In [14]:
from selenium import webdriver

In [15]:
from selenium.webdriver.common.keys import Keys

In [16]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as ec

In [17]:
from selenium.webdriver.support.ui import Select
import time
import os

In [45]:
def select_checkbox(checkbox):
    global driver
    if not checkbox.is_selected():
        driver.execute_script('arguments[0].click()', checkbox)

In [46]:
def fetch_results(driver):
    dir = r'C:\Users\Mirwi\Downloads'
    file = os.path.join(dir, 'CityWiseComparison.csv')
    title = ''
    
    for i in range(2,142):
        try:
            # GENERATE AN ID
            city_id = 'ReportViewer1_ctl00_ctl07_ctl03_ctl' + f'{i:02}'
            # SELECT CITY
            city = driver.find_element_by_id(city_id)
            select_checkbox(city)
            # SELECT ALL (YEAR)
            year = driver.find_element_by_id('ReportViewer1_ctl00_ctl03_ctl03_ctl00')
            select_checkbox(year)
            # SELECT ALL (MONTH)
            month = driver.find_element_by_id('ReportViewer1_ctl00_ctl05_ctl03_ctl00')
            select_checkbox(month)
            # SELECT ALL (CROPS)
            crops = driver.find_element_by_id('ReportViewer1_ctl00_ctl09_ctl03_ctl00')
            select_checkbox(crops)

            view_report = driver.find_element_by_id('ReportViewer1_ctl00_ctl00')
            
            city_label = driver.find_element_by_xpath(f'//label[@for="{city_id}"]')
            title = city_label.get_attribute('innerHTML')
            print('working on ', title)
            view_report.click()
            if WebDriverWait(driver,240).until(ec.visibility_of_all_elements_located(
                (By.ID, 'ReportViewer1_ctl01_ctl05_ctl00'))):
                
                export_select = Select(driver.find_element_by_id('ReportViewer1_ctl01_ctl05_ctl00'))
                export = driver.find_element_by_id('ReportViewer1_ctl01_ctl05_ctl01')
                # Get the title for the file
                name = title + '.csv'
                file_name = os.path.join(dir, name)
                # Keeps changing on page refresh makesure to update it
                export_select.select_by_value('CSV')

                window_before = driver.window_handles[0]
                export.click()
                print(f'Downloading {name}',end='')
                while True:
                    print('.',end='')
                    if os.path.isfile(file + '.part'):
                        time.sleep(10)
                    elif os.path.isfile(file):
                        break
                    else:
                        time.sleep(10)
                print("Renaming to file...")
                os.rename(file, file_name)
                print('Done with ', file_name)
                driver.switch_to.window(window_before)
                WebDriverWait(driver, 20).until(ec.title_contains("Untitled"))
        except TimeoutException as e:
            print('Timeout for file::',title)
        

In [47]:
profile = webdriver.FirefoxProfile()
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "text/csv")

driver = webdriver.Firefox(profile)
driver.get("http://www.amis.pk/Reports/CityWiseComparison.aspx")
print('starting')
driver.set_page_load_timeout(3600)
fetch_results(driver)
driver.quit()

starting
working on  Okara
Done with  C:\Users\Mirwi\Downloads\Okara.csv
working on  Attock
Done with  C:\Users\Mirwi\Downloads\Attock.csv
working on  Eminabad
Done with  C:\Users\Mirwi\Downloads\Eminabad.csv
working on  Badomalhi

KeyboardInterrupt: 